# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

# Import Data

In [2]:
path=r"C:\Users\chenh\z_Careerfoundry\A4_Instacart Basket Analysis"

customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

ords_prods_merge=pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'aggregated_ords_prods_merge.pkl'))

# STEP4 Wrangle the data

In [3]:
customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [3]:
# Rename columns
customers.rename(columns={'First Name' : 'first_name','Surnam' : 'last_name',
                          'Gender' : 'gender','STATE' : 'state','Age':'age','fam_status':'family_status'},inplace=True)

In [4]:
customers.head()

,user_id,first_name,last_name,gender,state,age,date_joined,n_dependants,family_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


# STEP5 Quality and Consistency

## Missing values

In [5]:
customers.isnull().sum()

user_id              0
first_name       11259
last_name            0
gender               0
state                0
age                  0
date_joined          0
n_dependants         0
family_status        0
income               0
dtype: int64

In [7]:
customers[customers['first_name'].isnull()]

,user_id,first_name,last_name,gender,state,age,date_joined,n_dependants,family_status,income
53,76659,NaN,Gilbert,Male,Colorado,26,1/1/2017,2,married,41709
73,13738,NaN,Frost,Female,Louisiana,39,1/1/2017,0,single,82518
82,89996,NaN,Dawson,Female,Oregon,52,1/1/2017,3,married,117099
99,96166,NaN,Oconnor,Male,Oklahoma,51,1/1/2017,1,married,155673
105,29778,NaN,Dawson,Female,Utah,63,1/1/2017,3,married,151819
...,...,...,...,...,...,...,...,...,...,...
206038,121317,NaN,Melton,Male,Pennsylvania,28,3/31/2020,3,married,87783
206044,200799,NaN,Copeland,Female,Hawaii,52,4/1/2020,2,married,108488
206090,167394,NaN,Frost,Female,Hawaii,61,4/1/2020,1,married,45275
206162,187532,NaN,Floyd,Female,California,39,4/1/2020,0,single,56325


Since information other than 'first_name' is meaningful, the null values are retained.

## Duplicated

In [6]:
customers.duplicated().sum()

0

No Duplicated rows

## Mixed-type

In [7]:
for col in customers.columns.tolist():
  weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customers[weird]) > 0:
    print (col)

first_name


first_name column has mixed data type. Let's fix it

In [8]:
customers['first_name'] = customers['first_name'].astype('str')

In [9]:
customers.dtypes

user_id           int64
first_name       object
last_name        object
gender           object
state            object
age               int64
date_joined      object
n_dependants      int64
family_status    object
income            int64
dtype: object

# STEP6 Combine Data

In [10]:
# datatype check
ords_prods_merge.dtypes

order_id                     int64
user_id                      int64
order_number                 int64
orders_day_of_week           int64
order_hour_of_day            int64
days_since_prior_order     float64
First_Buy                     bool
product_id                   int64
add_to_cart_order            int64
reordered                    int64
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
_merge                    category
price_range_loc             object
busiest_day                 object
busiest_days                object
busiest_period_of_day       object
max_order                    int64
loyalty_flag                object
average_spend              float64
spender_flag                object
Customer_frequency         float64
frequency_flag              object
dtype: object

The key columns are user_id and both are the same data type

In [11]:
# Drop the '_merge' column 
ords_prods_merge = ords_prods_merge.drop(columns=['_merge'])

In [13]:
ords_prods_customers_merge = ords_prods_merge.merge(customers,on='user_id',indicator=True)

# Export Data

In [15]:
ords_prods_customers_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_customers_aggregated.pkl'))